In [188]:
import re
import random
from hazm import *
import time
import copy
import numpy as np 
import pandas as pd
import pickle
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import defaultdict
from keras.layers import LSTM,Dropout,Dense,Input,Activation,TimeDistributed
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint

In [372]:
class LanguageModel():
    def __init__(self, lm_checkpoints, index2char, char2index, one_hot_encoder):
        self.index2char = index2char
        self.char2index = char2index
        self.one_hot_encoder = one_hot_encoder
        self.best_weights = lm_checkpoints
        self.model = self.lm_unit(lm_checkpoints)
        
    def lm_unit(self, weights):
        inputs = keras.Input(shape=(None,43), name = "X_data")
        layer, first_hidden_state, first_cell_state = LSTM(365,return_sequences=True,return_state = True)(inputs, initial_state=None)
        layer = Activation('elu')(layer)
        layer, second_hidden_state, second_cell_state = LSTM(365,return_sequences=True,return_state=True)(layer, initial_state=None)
        layer = Activation('elu')(layer)
        layer = Dense(43)(layer)
        prediction = Activation('softmax', name = 'y_pred')(layer)
        model = Model(inputs, [prediction,first_hidden_state, first_cell_state,second_hidden_state, second_cell_state])
        model.load_weights(weights)
        model.compile(optimizer="adam", loss={'y_pred':'categorical_crossentropy'})
        return model
    
    def get_next_states_and_output(self, prefix):
        string = "\n" + prefix
        vectorized = []
        for char in string:
            vectorized.append(self.one_hot_encoder[self.char2index[char]])
        vectorized = [vectorized]
        vectorized = np.array(np.array(vectorized))
        probabilities, hs1, cs1, hs2, cs2 = self.model.predict(vectorized)
        predicted_char = self.index2char[np.argmax(probabilities[0][-1])]
        return predicted_char
    
    def prefix_to_hiddens(self, prefix):
        string = "\n" + prefix
        vectorized = []
        for char in string:
            vectorized.append(self.one_hot_encoder[self.char2index[char]])
        vectorized = [vectorized]
        vectorized = np.array(np.array(vectorized))
        probabilities, hs1, cs1, hs2, cs2 = self.model.predict(vectorized)
        return hs2, cs2
    
    def get_next_char(self,probabilities):
        #chooses randomly between top k characters
        char_probabilities, k = probabilities[0][-1], 9
        top_k_indices  = np.argpartition(char_probabilities, -k)[-k:]
        top_k_probabilities = char_probabilities[np.argpartition(char_probabilities, -k)[-k:]]
        
        tmp, r = random.uniform(0,np.sum(top_k_probabilities)), 0
        for index, _ in enumerate(char_probabilities):
            if r + top_k_probabilities[index] < tmp:
                r += top_k_probabilities[index]
            else:
                return self.index2char[top_k_indices[index]]
            
        # chooses randomly between all characters
        #tmp, r = random.uniform(0,1), np.sum(top_k_probabilities)
        #for index, _ in enumerate(char_probabilities):
        #    if r + char_probabilities[index] < tmp:
        #        r += char_probabilities[index]
        #    else:
        #        return self.index2char[index]
    
    def generate_new_sample(self, prefix):
        generated_string = "\n" + prefix
        while generated_string[-1] != '\t' and len(generated_string)<300:
            vectorized = []
            for char in generated_string:
                vectorized.append(one_hot_encoder[char2index[char]])
            vectorized = [vectorized]
            vectorized = np.array(np.array(vectorized))
            probabilities, hs1, cs1, hs2, cs2 = model.predict(vectorized)
            predicted = self.get_next_char(probabilities)
           # predicted = index2char[np.argmax(probabilities[0][-1])]
            generated_string += predicted
            #print(generated_string)
            #print("================================")

        return generated_string
    
    def get_probability(self, prefix, predicting_last_character_prob = False):
        string = "\n" + prefix
        vectorized = []
        for char in string:
            vectorized.append(self.one_hot_encoder[self.char2index[char]])
        vectorized = [vectorized]
        vectorized = np.array(np.array(vectorized))
        probabilities, hs1, cs1, hs2, cs2 = self.model.predict(vectorized)
        index = np.argmax(probabilities[0][-1])
        predicted_char,char_prob = self.index2char[np.argmax(probabilities[0][-1])], probabilities[0][-1][index]
        if predicting_last_character_prob:
            if prefix != '':
                index_of_last_char = self.char2index[prefix[-1]]
                return predicted_char, char_prob,probabilities[0][-1][index_of_last_char]
            else:
                return predicted_char, char_prob, 1
        return predicted_char, char_prob
            

    def get_overall_probability(self, prefix):
        string, log_of_prefix_prob, strlen = "\n" + prefix, 0, len(prefix)-1
        for i in range(strlen):
            tmp1, tmp2 , prob = self.get_probability(prefix[:i],True)
            log_of_prefix_prob += np.log10(prob)
        return log_of_prefix_prob
    
    
    def get_data(self,direction):
        df = pd.DataFrame()
        df = pd.read_csv(direction,sep = "\t", error_bad_lines=False)
        df = df[['text','category']]
        df.fillna(value='نامحتوا', inplace=True)
        df = dict(df)
        texts= list(df['text'])
        return texts

    def normalize_document(self,content):
        not_valid_characters = r'[^بهگزارشودیخنجظمستحلعثصپکف;ئطض‌آق۲۵۰چء\^ذغ۱۷۹۶۸ژ۴۳1496327805؟. ‌]'
        normalizer = Normalizer()
        content = normalizer.normalize(content)
        content = re.sub('\$[^ا-ی]+',"",content)
        content = re.sub('\r\n',' ',content)
        content = re.sub('\d+/\d+|\d+\.\d+|\d+:\d+','^',content)
        content = re.sub(not_valid_characters,'',content)
        content = re.sub('[۰-۹]+',' N ',content)
        content = re.sub('[\d]+',' N ', content)
        content = re.sub('[\^]+',' N ',content)
        content = re.sub(' +', ' ', content)
        return content
    
    def clean(self,list_of_news):
        cleaned_news_list, total_char_tokens = [],0
        for content,index in zip(list_of_news,range(len(list_of_news))):
            normalized_content = self.normalize_document(content)
            total_char_tokens += len(normalized_content)
            cleaned_news_list.append(normalized_content)
        return cleaned_news_list, total_char_tokens/len(list_of_news)
    
    def get_cer_error(self, original_sentence, generated_sentence):
        original, generated = [x for x in original_sentence], [y for y in generated_sentence]
        N, M = len(original), len(generated)
        matrix = [[0 for j in range(N+1)] for i in range(M+1)]
        for j in range(N+1):
            matrix[0][j] = j
        for i in range(M+1):
            matrix[i][0] = i
        for i in range(1,M+1):
            for j in range(1,N+1):
                if original[j-1] == generated[i-1]:
                    matrix[i][j] = matrix[i-1][j-1]
                else:
                    matrix[i][j] = min(min(matrix[i-1][j],matrix[i][j-1]),matrix[i-1][j-1]) + 1 
        return ((matrix[M][N])/N)
            
    def evaluate(self, sentence):
        sentence = self.normalize_document(sentence)
        first_word = sentence.split(" ")[0]
        sentence = '\n' + sentence
        generated_sentence = self.generate_new_sample(first_word)
        generated_prob = self.get_overall_probability(generated_sentence)
        cer_error = self.get_cer_error(sentence,generated_sentence)
        #log_of_perplexity = self.nthRoot((np.log10(1) - generated_prob),len(generated_sentence))
        perplexity = np.power((np.log10(1) - generated_prob),(1/len(generated_sentence)))
        return generated_sentence, cer_error, perplexity
    
        
        
        
        
            

In [299]:
char2index,index2char = None, None
with open("char2index.pickle",'rb') as f:
    char2index = pickle.load(f)
with open("index2char.pickle",'rb') as f:
    index2char = pickle.load(f)
one_hot_encoder = [[0 for i in range(len(char2index))] for j in range(len(char2index))]
for i, row in enumerate(char2index):
    one_hot_encoder[i][i] = 1
    one_hot_encoder[i] = np.asarray(one_hot_encoder[i])

In [373]:
lm = LanguageModel('weights-improvement-04-1.1540.hdf5', index2char, char2index,one_hot_encoder)

In [321]:
print(lm.get_next_states_and_output('امروز بهار دست جم'))

ع


In [302]:
print(lm.prefix_to_hiddens('امروز بهار دست جم'))

(array([[-1.98573172e-02,  1.05035797e-01, -9.98584449e-01,
        -1.60600320e-02,  2.12307692e-01,  8.09088945e-01,
        -7.59041519e-04,  7.50950336e-01, -4.55635905e-01,
         5.59452951e-01, -9.93418932e-01,  5.77428602e-02,
         7.08596885e-01,  1.42902955e-01, -2.94912793e-03,
        -2.62129307e-03, -1.06797814e-02, -8.83022785e-01,
         2.30936706e-02, -3.82587671e-01, -7.66082406e-01,
        -6.47640705e-01,  8.67231727e-01,  7.71092892e-01,
        -8.81958544e-01,  3.67537707e-01, -2.15924516e-01,
        -3.07725936e-01, -7.00035036e-01, -2.60618865e-01,
         7.58502960e-01,  6.51420474e-01, -5.27965248e-01,
         9.94798601e-01, -9.31304470e-02,  5.08151669e-03,
        -6.94135666e-01, -7.49167800e-01, -2.12361977e-01,
        -3.31532647e-04, -9.52448070e-01,  4.11935849e-03,
         2.45454371e-01,  1.45101547e-03, -9.92581367e-01,
         9.92699087e-01,  4.81272042e-02,  4.66837585e-02,
        -8.63666356e-01,  3.25680175e-03,  7.64549851e-

In [259]:
print(lm.generate_new_sample('باشگاه پرسپولیس'))


باشگاه پرسپولیس 

باشگاه پرسپولیس ب

باشگاه پرسپولیس بر

باشگاه پرسپولیس برا

باشگاه پرسپولیس برای

باشگاه پرسپولیس برای 

باشگاه پرسپولیس برای ا

باشگاه پرسپولیس برای ای

باشگاه پرسپولیس برای این

باشگاه پرسپولیس برای این 

باشگاه پرسپولیس برای این ک

باشگاه پرسپولیس برای این کا

باشگاه پرسپولیس برای این کار

باشگاه پرسپولیس برای این کار 

باشگاه پرسپولیس برای این کار ب

باشگاه پرسپولیس برای این کار بر

باشگاه پرسپولیس برای این کار برا

باشگاه پرسپولیس برای این کار برای

باشگاه پرسپولیس برای این کار برای 

باشگاه پرسپولیس برای این کار برای ت

باشگاه پرسپولیس برای این کار برای تم

باشگاه پرسپولیس برای این کار برای تما

باشگاه پرسپولیس برای این کار برای تمام

باشگاه پرسپولیس برای این کار برای تمام 

باشگاه پرسپولیس برای این کار برای تمام م

باشگاه پرسپولیس برای این کار برای تمام مس

باشگاه پرسپولیس برای این کار برای تمام مسا

باشگاه پرسپولیس برای این کار برای تمام مساب

باشگاه پرسپولیس برای این کار برای تمام مسابق

باشگاه پرسپولیس برای این کار برای تمام مسابقا

باشگاه پرسپولیس برای این


باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . ای

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این 

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این ن

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نو

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نوی

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نویس

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نویسن

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین 


باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نویسنده در بیست و چهارم مشهد به عنوان یک دیپلم از 

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نویسنده در بیست و چهارم مشهد به عنوان یک دیپلم از ت

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نویسنده در بیست و چهارم مشهد به عنوان یک دیپلم از تی

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نویسنده در بیست و چهارم مشهد به عنوان یک دیپلم از تیم

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نویسنده در بیست و چهارم مشهد به عنوان یک دیپلم از تیمو

باشگاه پرسپولیس برای این کار برای تمام مسابقاتی که بخشی از این افراد نیز برای تمیدر بهترین راه می‌گذارد . این نویسنده در بیست و چهارم مشهد به عنوان یک دیپلم از تیمور

باشگاه پرسپو

In [291]:
print(lm.get_probability('منچستریو'))

('ن', 0.7921135)


In [293]:
print(lm.get_overall_probability('منچستریونایتد'))

-44.585886001586914


In [295]:
print(lm.get_overall_probability('پرسپولیس'))

-24.055863857269287


In [314]:
a = time.time()
sentence, cer, perp = lm.evaluate('پرسپولی')
b = time.time()
print(b-a)

generated
got probability
error calculated
perplexity calculated
89.43688797950745


In [315]:
sentence

'\n\nپرسپولیس از موضوع کنفدراسیون ایرانی در پیامی در نشست خبری که در سال N با اجرای طرح مهار تعیین\u200cکیم در مقابل ملی پوش این کودکان به درخواست کارشناسی مستند ارزیابی شد تا این دیدار که به دلیل اینکه به صورت افزایش نارسایی\u200cهای مردم در منطقه برخی از برنامه\u200cهای دولت آمریکا هستند و تنها در مجلس را منتشر می\u200cکند و همان طور که در این زمینه دسترسی به ارائه توانمندی\u200cهای اجتماعی و همچنین این اجلاس مسائل است . رئیس جمهور ایران در پایان گفت بی\u200cگیاه نظام جمعیت درگیر دولت آمریکا وی با رئیس جمهور این کشور نشان است مذاکره\u200cکنندگانی که بدون ما به دنبال ماندگاری در سوریه برخوردارند تا آنچه می\u200cگویند تا پایان دیدار با'

In [317]:
cer

74.0

In [318]:
perp

1.013728583584183

In [361]:
texts = lm.get_data('test.csv')
cleaned_news,avg = lm.clean(texts)

In [374]:
generated_news, errors, perps = [],[],[]
for index,document in enumerate(cleaned_news):
    a = time.time()
    g, e, p = lm.evaluate(document)
    generated_news.append(g)
    errors.append(e)
    perps.append(p)
    b = time.time()
    print(index,":",b-a)


0 : 32.88781690597534
1 : 31.016573190689087
2 : 31.011428117752075
3 : 31.631128072738647
4 : 33.10249185562134
5 : 31.067103147506714
6 : 31.368587970733643
7 : 32.02632808685303
8 : 31.764692068099976
9 : 0.47966814041137695
10 : 5.356243848800659
11 : 31.33312702178955
12 : 30.925628900527954
13 : 0.27959203720092773
14 : 6.6695568561553955
15 : 31.462249040603638
16 : 0.47537994384765625
17 : 31.318060874938965
18 : 0.26747703552246094
19 : 0.48732900619506836
20 : 31.26533794403076
21 : 31.09360694885254
22 : 31.1686589717865
23 : 31.53720188140869
24 : 31.01773476600647
25 : 30.89380407333374
26 : 31.901878118515015
27 : 31.577480792999268
28 : 30.90929102897644
29 : 31.60162591934204
30 : 0.531653881072998
31 : 31.86600399017334
32 : 34.099059104919434
33 : 33.72629404067993
34 : 33.850669145584106
35 : 34.426100969314575
36 : 31.811744928359985
37 : 0.47968387603759766
38 : 30.971148014068604
39 : 31.433274030685425
40 : 31.11286187171936
41 : 30.94097590446472
42 : 31.5249001

337 : 21.408673763275146
338 : 31.36720895767212
339 : 32.150423765182495
340 : 31.356996059417725
341 : 31.08182191848755
342 : 31.00177311897278
343 : 31.146950244903564
344 : 22.24804997444153
345 : 8.367920160293579
346 : 31.03003978729248
347 : 32.01422309875488
348 : 31.63752508163452
349 : 16.85453701019287
350 : 32.054471015930176
351 : 31.008225202560425
352 : 31.120877981185913
353 : 31.920030117034912
354 : 31.129247903823853
355 : 0.46921706199645996
356 : 0.5180830955505371
357 : 31.5523579120636
358 : 31.106953144073486
359 : 31.714145183563232
360 : 31.30952501296997
361 : 0.270413875579834
362 : 31.135642766952515
363 : 30.972465991973877
364 : 31.761756896972656
365 : 21.76690411567688
366 : 24.748621225357056
367 : 6.577676057815552
368 : 11.8746178150177
369 : 7.021044969558716
370 : 31.198052883148193
371 : 13.46120309829712
372 : 31.924174785614014
373 : 0.4921119213104248
374 : 0.48030686378479004
375 : 0.5008740425109863
376 : 0.5578629970550537
377 : 14.27383804

669 : 31.418998956680298
670 : 31.321241855621338
671 : 31.58272409439087
672 : 30.738298177719116
673 : 31.580490112304688
674 : 31.1584951877594
675 : 31.314989805221558
676 : 30.843036890029907
677 : 30.928597927093506
678 : 31.303297758102417
679 : 5.395943880081177
680 : 31.03866696357727
681 : 30.900150060653687
682 : 31.53223991394043
683 : 31.209025144577026
684 : 31.421241760253906
685 : 31.33321499824524
686 : 0.47926878929138184
687 : 27.05863904953003
688 : 31.403740882873535
689 : 31.148525953292847
690 : 31.06831979751587
691 : 31.52246594429016
692 : 31.20223093032837
693 : 5.876916885375977
694 : 30.952956914901733
695 : 32.03925919532776
696 : 31.14899468421936
697 : 8.823559284210205
698 : 0.6411099433898926
699 : 32.368627309799194
700 : 31.022372245788574
701 : 31.239994764328003
702 : 31.858284950256348
703 : 31.847546339035034
704 : 31.0701642036438
705 : 0.2608046531677246
706 : 0.47659993171691895
707 : 16.58570408821106
708 : 0.4976189136505127
709 : 31.2571787

KeyboardInterrupt: 

In [375]:
with open("new_samples.txt", 'w') as f:
    for sample in generated_news:
        f.write(sample)

In [378]:
probabilities = []
for index,perplexity in enumerate(perps):
    probabilities.append(1.0/(np.power(perplexity,len(generated_news[index]))))

In [380]:
with open("probabilities.txt", 'w') as f:
    for probability in probabilities:
        f.write(sample)